# Task 1: Create a Prescription Parser using CRF
This task tests your ability to build a Doctor Prescription Parser with the help of CRF model

Your job is to build a Prescription Parser that takes a prescription (sentence) as an input and find / label the words in that sentence with one of the already pre-defined labels

### Problem: SEQUENCE PREDICTION - Label words in a sentence
#### Input : Doctor Prescription in the form of a sentence split into tokens
- Ex: Take 2 tablets once a day for 10 days

#### Output : FHIR Labels
- ('Take', 'Method')
- ('2', 'Qty') 
- ('tablets', 'Form')
- ('once', 'Frequency')
- ('a', 'Period') 
- ('day', 'PeriodUnit')
- ('for', 'FOR')
- ('10', 'Duration')
- ('days', 'DurationUnit') 

### Major Steps
- Install necessary library
- Import the libraries
- Create training data with labels
    - Split the sentence into tokens
    - Compute POS tags
    - Create triples
- Extract features
- Split the data into training and testing set
- Create CRF model
- Save the CRF model
- Load the CRF model
- Predict on test data
- Accuracy

#### Install necesaary library

In [27]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/daniyarkurmanbayev/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

#### Import the necessary libraries

In [164]:
from typing import List
from nltk import pos_tag
from sklearn_crfsuite import CRF, metrics
import joblib
import random

### Input data (GIVEN)
#### Creating the inputs to the ML model in the following form:
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [4]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [5]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s): 
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [22]:
def tuples_maker(inp: List[str], out: List[str]):
    if len(inp) != len(out):
        raise "Lengths not equal"
    sample_data = []
    for i in range(0, len(inp)):
        sublist = []
        for j in range(0, len(inp[i])):
            sublist.append((inp[i][j], out[i][j]))
        sample_data.append(sublist)
    return sample_data

In [21]:
tuples_maker(input_sigs, output_labels)

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')],
 [('inject', 'Method'), ('2', 'Qty'), ('units', 'Form')],
 [('x', 'FOR'), ('2', 'Duration'), ('weeks', 'DurationUnit')],
 [('x', 'FOR'), ('3', 'Duration'), ('days', 'DurationUnit')],
 [('every', 'EVERY'), ('day', 'Period')],
 [('every', 'EVERY'), ('2', 'Period'), ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'), ('3', 'Period'), ('days', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('1', 'Period'),
  ('to', 'TO'),
  ('2', 'PeriodMax'),
  ('months', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('2', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('4', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('days', 'PeriodUnit')],
 [('take', 'Method'),
  ('two', 'Qty'),
  ('to', 'TO'),
  ('four', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('2', 'Qty'),
  ('to', 'TO'),
  ('4', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('3', 

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output: 
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [57]:
def triples_maker(whole_data):
    new_data = []
    for sentence in whole_data:
        new_sentence = []
        for word in sentence:
            new_word = (word[0], pos_tag([word[0]])[0][1], word[1])
            new_sentence.append(new_word)
        new_data.append(new_sentence)
    return new_data

In [58]:
whole_data = tuples_maker(input_sigs, output_labels)
sample_data = triples_maker(whole_data)
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'NN', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'NN', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'NN', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
- SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag
#### Feel free to include more features

In [59]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [65]:
def token_to_label(doc, i):
    return doc[i][-1]

### Running the feature extractor on the training data 
- Feature extraction
- Train-test-split

In [166]:
def train_test_split(data, split=0.2, random_state=42):
    def apply(func, data):
        return [
            [
                func(sentence, index) 
                for index in range(0, len(sentence))
            ] 
            for sentence in data
        ]
    
    random.seed(random_state)
    test = random.sample(data, int(len(data) * split))
    train = [sentence for sentence in data if not test.__contains__(sentence)]
    return apply(token_to_features, train), apply(token_to_label, train), apply(token_to_features, test), apply(token_to_label, test)


In [172]:
X_train, y_train, X_test, y_test = train_test_split(sample_data, split=0.1)

### Training the CRF model with the features extracted using the feature extractor method

In [173]:
crf = CRF(algorithm='lbfgs', c1=0.2, c2=10, max_iterations=50)
crf.fit(X_train, y_train)
joblib.dump(crf, 'my_crf.joblib') 

['my_crf.joblib']

### Predicting the test data with the built model

In [175]:
print(metrics.flat_classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

       AFTER      0.000     0.000     0.000         1
       EVERY      1.000     1.000     1.000         1
        Form      0.800     1.000     0.889         4
   Frequency      0.500     1.000     0.667         1
      Method      1.000     1.000     1.000         4
      Period      1.000     1.000     1.000         4
   PeriodMax      0.000     0.000     0.000         1
  PeriodUnit      1.000     0.750     0.857         4
         Qty      0.800     1.000     0.889         4
          TO      1.000     1.000     1.000         1

    accuracy                          0.880        25
   macro avg      0.710     0.775     0.730        25
weighted avg      0.836     0.880     0.848        25



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

### Putting all the prediction logic inside a predict method

In [195]:
def predict(sig):
    def to_tokenized_pos(sig):
        words = sig.split()
        return [(word, pos_tag([word])[0][1]) for word in words]

    def tokens_to_features(doc):
        all_features = []
        for i, unit in enumerate(doc):
            word = unit[0]
            postag = unit[1]

            # Common features for all words
            features = [
                'bias',
                'word.lower=' + word.lower(),
                'word[-3:]=' + word[-3:],
                'word[-2:]=' + word[-2:],
                'word.isupper=%s' % word.isupper(),
                'word.istitle=%s' % word.istitle(),
                'word.isdigit=%s' % word.isdigit(),
                'postag=' + postag
            ]

            # Features for words that are not
            # at the beginning of a document
            if i > 0:
                word1 = doc[i-1][0]
                postag1 = doc[i-1][1]
                features.extend([
                    '-1:word.lower=' + word1.lower(),
                    '-1:word.istitle=%s' % word1.istitle(),
                    '-1:word.isupper=%s' % word1.isupper(),
                    '-1:word.isdigit=%s' % word1.isdigit(),
                    '-1:postag=' + postag1
                ])
            else:
                # Indicate that it is the 'beginning of a document'
                features.append('BOS')

            # Features for words that are not
            # at the end of a document
            if i < len(doc)-1:
                word1 = doc[i+1][0]
                postag1 = doc[i+1][1]
                features.extend([
                    '+1:word.lower=' + word1.lower(),
                    '+1:word.istitle=%s' % word1.istitle(),
                    '+1:word.isupper=%s' % word1.isupper(),
                    '+1:word.isdigit=%s' % word1.isdigit(),
                    '+1:postag=' + postag1
                ])
            else:
                # Indicate that it is the 'end of a document'
                features.append('EOS')

            all_features.append(features)
        return all_features

    crf_model = joblib.load('my_crf.joblib')
    tokenized_pos = to_tokenized_pos(sig)
    features = tokens_to_features(tokenized_pos)
    predictions = crf_model.predict_single(features)
    print(sig)
    print(predictions)
    return predictions

### Sample predictions

In [196]:
predictions = predict("take 2 tabs every 6 hours x 10 days")

take 2 tabs every 6 hours x 10 days
['Method', 'Qty', 'Form', 'EVERY', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']


In [197]:
predictions = predict("2 capsu for 10 day at bed")

2 capsu for 10 day at bed
['Qty', 'Form', 'FOR', 'Duration', 'Form', 'AT', 'WHEN']


In [198]:
predictions = predict("2 capsu for 10 days at bed")

2 capsu for 10 days at bed
['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']


In [199]:
predictions = predict("5 days 2 tabs at bed")

5 days 2 tabs at bed
['Qty', 'Form', 'Qty', 'Form', 'AT', 'WHEN']


In [200]:
predictions = predict("3 tabs qid x 10 weeks")

3 tabs qid x 10 weeks
['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit']


In [202]:
predictions = predict("x 30 days")

x 30 days
['Method', 'Qty', 'DurationUnit']


In [203]:
predictions = predict("x 20 months")

x 20 months
['Method', 'Qty', 'Form']


In [204]:
predictions = predict("take 2 tabs po tid for 10 days")

take 2 tabs po tid for 10 days
['Method', 'Qty', 'Form', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']


In [205]:
predictions = predict("take 2 capsules po every 6 hours")

take 2 capsules po every 6 hours
['Method', 'Qty', 'Form', 'Frequency', 'EVERY', 'Period', 'PeriodUnit']


In [206]:
predictions = predict("inject 2 units pu tid")

inject 2 units pu tid
['Method', 'Qty', 'Form', 'Period', 'PeriodUnit']


In [207]:
predictions = predict("swallow 3 caps tid by mouth")

swallow 3 caps tid by mouth
['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit']


In [208]:
predictions = predict("inject 3 units orally")

inject 3 units orally
['Method', 'Qty', 'Form', 'PeriodUnit']


In [209]:
predictions = predict("orally take 3 tabs tid")

orally take 3 tabs tid
['Qty', 'Method', 'Qty', 'Form', 'PeriodUnit']


In [210]:
predictions = predict("by mouth take three caps")

by mouth take three caps
['Qty', 'Form', 'Method', 'Qty', 'Form']


In [211]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")

take 3 tabs orally three times a day for 10 days at bedtime
['Method', 'Qty', 'Form', 'Method', 'Qty', 'Form', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']


In [212]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")

take 3 tabs orally bid for 10 days at bedtime
['Method', 'Qty', 'Form', 'Frequency', 'Period', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']


In [213]:
predictions = predict("take 3 tabs bid orally at bed")

take 3 tabs bid orally at bed
['Method', 'Qty', 'Form', 'Qty', 'Form', 'AT', 'WHEN']


In [214]:
predictions = predict("take 10 capsules by mouth qid")

take 10 capsules by mouth qid
['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit']


In [215]:
predictions = predict("inject 10 units orally qid x 3 months")

inject 10 units orally qid x 3 months
['Method', 'Qty', 'Form', 'Frequency', 'Period', 'Method', 'Qty', 'Form']


In [216]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")

please take 2 tablets per day for a month in the morning and evening each day
['Qty', 'Method', 'Qty', 'Form', 'Period', 'PeriodUnit', 'FOR', 'Period', 'PeriodUnit', 'Qty', 'Form', 'Period', 'Qty', 'Form', 'Period', 'PeriodUnit']


In [217]:
prediction = predict("Amoxcicillin QID 30 tablets")

Amoxcicillin QID 30 tablets
['Method', 'Method', 'Qty', 'Form']


In [218]:
prediction = predict("take 3 tabs TID for 90 days with food")

take 3 tabs TID for 90 days with food
['Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'Period', 'PeriodUnit']


In [219]:
prediction = predict("with food take 3 tablets per day for 90 days")

with food take 3 tablets per day for 90 days
['Qty', 'Form', 'Method', 'Qty', 'Form', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']


In [220]:
prediction = predict("with food take 3 tablets per week for 90 weeks")

with food take 3 tablets per week for 90 weeks
['Qty', 'Form', 'Method', 'Qty', 'Form', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']


In [221]:
prediction = predict("take 2-4 tabs")

take 2-4 tabs
['Method', 'Qty', 'Form']


In [222]:
prediction = predict("take 2 to 4 tabs")

take 2 to 4 tabs
['Method', 'Qty', 'Form', 'Qty', 'Form']


In [223]:
prediction = predict("take two to four tabs")

take two to four tabs
['Method', 'Qty', 'Form', 'Qty', 'Form']


In [224]:
prediction = predict("take 2-4 tabs for 8 to 9 days")

take 2-4 tabs for 8 to 9 days
['Method', 'Qty', 'Form', 'FOR', 'Duration', 'TO', 'Qty', 'DurationUnit']


In [225]:
prediction = predict("take 20 tabs every 6 to 8 days")

take 20 tabs every 6 to 8 days
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Qty', 'DurationUnit']


In [226]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Qty', 'DurationUnit']


In [227]:
prediction = predict("take 2 tabs every 2 to 10 weeks")

take 2 tabs every 2 to 10 weeks
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Qty', 'Form']


In [228]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Qty', 'DurationUnit']


In [229]:
prediction = predict("take 2 tabs every 2 to 10 months")

take 2 tabs every 2 to 10 months
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Qty', 'Form']


In [230]:
prediction = predict("every 60 mins")

every 60 mins
['EVERY', 'Qty', 'Form']


In [231]:
prediction = predict("every 10 mins")

every 10 mins
['EVERY', 'Qty', 'Form']


In [232]:
prediction = predict("every two to four months")

every two to four months
['EVERY', 'Period', 'TO', 'Qty', 'Form']


In [233]:
prediction = predict("take 2 tabs every 3 to 4 days")

take 2 tabs every 3 to 4 days
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Qty', 'DurationUnit']


In [234]:
prediction = predict("every 3 to 4 days take 20 tabs")

every 3 to 4 days take 20 tabs
['EVERY', 'Period', 'TO', 'Qty', 'Form', 'Method', 'Qty', 'Form']


In [235]:
prediction = predict("once in every 3 days take 3 tabs")

once in every 3 days take 3 tabs
['Qty', 'Form', 'EVERY', 'Qty', 'Form', 'Method', 'Qty', 'Form']


In [236]:
prediction = predict("take 3 tabs once in every 3 days")

take 3 tabs once in every 3 days
['Method', 'Qty', 'Form', 'Frequency', 'Period', 'EVERY', 'Qty', 'DurationUnit']


In [237]:
prediction = predict("orally take 20 tabs every 4-6 weeks")

orally take 20 tabs every 4-6 weeks
['Qty', 'Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']


In [238]:
prediction = predict("10 tabs x 2 days")

10 tabs x 2 days
['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']


In [239]:
prediction = predict("3 capsule x 15 days")

3 capsule x 15 days
['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']


In [240]:
prediction = predict("10 tabs")

10 tabs
['Qty', 'Form']
